<h1>Forecasting a Time Series

We aim to forecast the value at the next time step

Several examples of Time series :
- Daily temperature of a city
- Company's financial health
- Stock market

In [33]:
import numpy as np
from tensorflow import keras
import tensorflow as tf

Let's generate a time series

In [9]:
def generate_time_series(batch_size, n_steps):
    """Generate a time series"""
    #Create 4 arrays of (batch_size) rows
    freq1, freq2, offsets1, offsets2 = np.random.rand(4, batch_size, 1)
    #Time is an array of evenly spaced times between 0 and 1
    time = np.linspace(0,1, n_steps)
    
    series = 0.5* np.sin((time - offsets1)* (freq1*10 + 10))#wave 1
    series += 0.4 * np.sin((time - offsets2) * (freq2 * 20 + 20)) # + wave 2
    series += 0.3 * (np.random.rand(batch_size, n_steps) - 0.5) # + noise
    
    return series[..., np.newaxis].astype(np.float32)

Batch size creates as many time series as requested each of length n_steps

- X_trains : 7000 times series [7000,50,1]
- X_valid : 2000 time series [2000, 50, 1]
- X_test : 1000 time series [1000, 50, 1]

In [12]:
n_steps = 50
series = generate_time_series(10000, n_steps +1)
X_train, y_train = series[:7000, :n_steps], series[:7000, -1]
X_valid, y_valid = series[7000:9000, :n_steps], series[7000:9000, -1]
X_test, y_test = series[9000:, :n_steps], series[9000:, -1]

Baseline metrics : Let's test with a fully connected network

In [31]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape = [50,1]),
    keras.layers.Dense(1)
])
model.compile(loss = 'mse', optimizer='adam')
model.fit(X_train,y_train, epochs = 20, validation_data=(X_valid, y_valid), batch_size=32)

Epoch 1/20
219/219 [==============================] - 0s 1ms/step - loss: 0.1099 - val_loss: 0.0456
Epoch 2/20
219/219 [==============================] - 0s 928us/step - loss: 0.0399 - val_loss: 0.0349
Epoch 3/20
219/219 [==============================] - 0s 949us/step - loss: 0.0334 - val_loss: 0.0315
Epoch 4/20
219/219 [==============================] - 0s 973us/step - loss: 0.0304 - val_loss: 0.0294
Epoch 5/20
219/219 [==============================] - 0s 922us/step - loss: 0.0281 - val_loss: 0.0279
Epoch 6/20
219/219 [==============================] - 0s 929us/step - loss: 0.0265 - val_loss: 0.0260
Epoch 7/20
219/219 [==============================] - 0s 966us/step - loss: 0.0254 - val_loss: 0.0250
Epoch 8/20
219/219 [==============================] - 0s 991us/step - loss: 0.0243 - val_loss: 0.0242
Epoch 9/20
219/219 [==============================] - 0s 922us/step - loss: 0.0234 - val_loss: 0.0235
Epoch 10/20
219/219 [==============================] - 0s 933us/step - loss: 0.0227 

We obtain an MSE of about 0.02

Let's see if we can beat that with a simple RNN

In [36]:
np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([
    #We don't need to specify the length of the inpu sequences, sin a RNN can process any number of time steps
    keras.layers.SimpleRNN(1, input_shape=[None, 1])
])
model.compile(loss = 'mse', optimizer='adam')
model.fit(X_train,y_train, epochs = 20, validation_data=(X_valid, y_valid), batch_size=32)

Epoch 1/20
219/219 [==============================] - 2s 7ms/step - loss: 0.2750 - val_loss: 0.1906
Epoch 2/20
219/219 [==============================] - 2s 7ms/step - loss: 0.1666 - val_loss: 0.1383
Epoch 3/20
219/219 [==============================] - 1s 7ms/step - loss: 0.1326 - val_loss: 0.1167
Epoch 4/20
219/219 [==============================] - 1s 7ms/step - loss: 0.1135 - val_loss: 0.1005
Epoch 5/20
219/219 [==============================] - 1s 7ms/step - loss: 0.0990 - val_loss: 0.0882
Epoch 6/20
219/219 [==============================] - 1s 7ms/step - loss: 0.0876 - val_loss: 0.0784
Epoch 7/20
219/219 [==============================] - 1s 7ms/step - loss: 0.0788 - val_loss: 0.0710
Epoch 8/20
219/219 [==============================] - 1s 7ms/step - loss: 0.0716 - val_loss: 0.0650
Epoch 9/20
219/219 [==============================] - 1s 7ms/step - loss: 0.0659 - val_loss: 0.0601
Epoch 10/20
219/219 [==============================] - 1s 7ms/step - loss: 0.0612 - val_loss: 0.0562

Let's try to add more recurrent layers

In [37]:
model = keras.models.Sequential([
    #return_sequences = True will output a 3D array containing all time steps
    keras.layers.SimpleRNN(20, return_sequences = True, input_shape=[None,1]),
    keras.layers.SimpleRNN(20, return_sequences=True),
    keras.layers.SimpleRNN(1)
])

model.compile(loss = 'mse', optimizer='adam')
model.fit(X_train,y_train, epochs = 20, validation_data=(X_valid, y_valid), batch_size=32)

Epoch 1/20
219/219 [==============================] - 4s 17ms/step - loss: 0.0533 - val_loss: 0.0274
Epoch 2/20
219/219 [==============================] - 4s 16ms/step - loss: 0.0252 - val_loss: 0.0247
Epoch 3/20
219/219 [==============================] - 4s 17ms/step - loss: 0.0235 - val_loss: 0.0248
Epoch 4/20
219/219 [==============================] - 4s 16ms/step - loss: 0.0228 - val_loss: 0.0230
Epoch 5/20
219/219 [==============================] - 4s 18ms/step - loss: 0.0221 - val_loss: 0.0217
Epoch 6/20
219/219 [==============================] - 4s 16ms/step - loss: 0.0218 - val_loss: 0.0252
Epoch 7/20
219/219 [==============================] - 3s 16ms/step - loss: 0.0214 - val_loss: 0.0214
Epoch 8/20
219/219 [==============================] - 4s 16ms/step - loss: 0.0208 - val_loss: 0.0221
Epoch 9/20
219/219 [==============================] - 4s 16ms/step - loss: 0.0204 - val_loss: 0.0203
Epoch 10/20
219/219 [==============================] - 4s 16ms/step - loss: 0.0197 - val_lo